# Guide to Running

## Pre Processing

Here I am doing some pre-processing on the raw datasets to make them more usable for downstream analysis.  


This stuff is pretty computationally intensive.  If you want to run this part do the following things:  
* Ask yourself if you really need to do this or if you can start with the data already processed (I can provide this in a nice small binary file which is much easier to handle). 
* Get yourself a pretty high memory machine. You are going to need to do quantile-normalization on about 2000 450k arrays.  This took up about 100GB in memory.  You could probably figure out how to do this out of memory but its only really a one time thing so probably not necessary. 
* Download the raw EPIC dataset from GEO [link](http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE51032).  
* Contact me for the raw dataset from Hannum et al. (This data is on GEO but not in the most raw form... I will try and fix this soon)

[__Metylation Normalization Protocol__](./PreProcessing/Methylation_Normalization_MINFI.ipynb)  
This mainly uses functions provided in the [R bioconductor minfi package](http://bioconductor.org/packages/release/bioc/html/minfi.html). 
* Read in all of the raw .IDAT methylation files for our datasets as well as the two controls 
* Extract detection p-values for all probes across all samples
* Run quantile normalization on the concatinated dataset
* Run estimateCellCounts on the concatinated dataset
* Save the results of this analysis 

#### Saving things in HDF5 Format

These 450k arrays can get pretty unweildy due to their size when trying to do in-memory data analysis.  To help out with this and generally speed up I/O I like to store these data as [Pandas HDF5](http://pandas.pydata.org/pandas-docs/stable/io.html#hdf5-pytables) data objects.  This data format has the advantage of being much faster than handling everything in .csvs like you would generally do with expression datasets.

* [__save detection p values__](./PreProcessing/save_detection_p_values.ipynb) 
* [__compile probe annotations__](./PreProcessing/Compile_Probe_Annotations.ipynb) 
* [__Pan Study Data__](./PreProcessing/Pan_Study_Data.ipynb)

#### BMIQ Processing

We run quantile normalization in the minfi processing pipeline but it has been recomended to also run BMIQ on this normalized data.  In addition the processing protocol for use of the Horvath aging model requires a modified verision of this.  This is done for various datasets in the following notebooks. 

* [__BMIQ Horvath__](./PreProcessing/BMIQ_Horvath.ipynb) 
* [__BMIQ Normalization__](./PreProcessing/BMIQ_Normalization.ipynb)

## Running Parallel Jobs for Linear Models

I have created various scripts for running linear models on the methylation data.  With about a half of a million probes on the chip, this can take a while to do in series so I use a relatively primative map-reduce paradigm. 
* A dataset is saved in 100 chunks in HDF5 format
* Covariates are saved in the same data-store  
* A python script to run the linear models is hard coded with the table chunk number as an input parameter
* An SGE driver script is created to run a parallel job 
* The parallel job is run on the cluster 
* The results are merged back together on the local machine  

In the [Parallel](./Parallel) folder, I have a number of notebooks which contain the hard coded python script and the code for generating the SGE driver. Our cluster shares its filesystem with our network storage, so for applying this to different systems some modifications to the protocol are likely needed. 

## Analysis Setup Notebooks

These are notebooks which I generally run as upstream dependencies of other notebooks, think of them as python modules. I include these as notebook rather than modules because there are some global variables ad-hoc decisions being invoked and I do not think it is appropriate to abstract away into modules. Is this the best software development process? Likely not, but this is data-analysis and this is also grad-ware, both of which have a certain level of sloppiness which is more or less unavoidable. 
* [__Imports__](./Setup/Imports.ipynb)  
Imports packages into global namespace, sets up a few helper functions, and global path variables.  __If you are running this yourself you must [change the global parameters here](Setup/Imports.ipynb#Global-Parameters)__. 
* [__DX Imports__](./Setup/DX_Imports.ipynb)  
Sets up some helper functions for doing analysis of differential methylation and reads in probe annotations for methylation arrays. 
* [__Methylation Age Models__](./Setup/MethylationAgeModels.ipynb)  
Reads in the linear data for the two epigenetic aging models and creates helper functions to apply each model across a set of patients. 
* [__Read HIV Data__](./Setup/Read_HIV_Data.ipynb)  
This reads in the methylation data for our primary case-control cohort as well a lot of associated clinical data.  I also do a little pre-filtering based on basic selection criteria (age range, HAART status, gender).

## Main Data Analysis

Here is where I do the main data analysis for the manuscript.  
* [__HIV Age Advancment__](./HIV_Age_Advancement.ipynb)  
Here I read in the data and run the methylation age models.  
* [__HIV Age Advancement: Confounders__](./HIV_Age_Advancement_Confounders.ipynb)  
Here I am looking at confounding from patients' blood composition as well as association of age advancment with other clinical variables that we have available.